In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('clean_Movie5')

In [4]:
df

,Unnamed: 0,sentiment,Reviews3
0,0,Positive,stuff going moment mj started listening music ...
1,1,Positive,nothe classic war worlds timothy hines enterta...
2,2,Negative,starts manager nicholas bell giving welcome in...
3,3,Negative,must assumed praised nothe greatest filmed ope...
4,4,Positive,superbly trashy wondrously unpretentious explo...
...,...,...,...
24995,24995,Negative,seems like consideration gone imdb reviews wen...
24996,24996,Negative,not believe made completely unnecessary first ...
24997,24997,Negative,guy loser not get girls needs build picked str...
24998,24998,Negative,minute documentary buñuel made early spain poo...


In [3]:
df['Label'] = df['sentiment'].apply(lambda x: '1' if x == 'Positive' else ('0' if x == 'Negative' else 'neutral'))

In [4]:
df.drop(['Unnamed: 0','sentiment'],inplace=True,axis=1)

In [7]:
df

,Reviews3,Label
0,stuff going moment mj started listening music ...,1
1,nothe classic war worlds timothy hines enterta...,1
2,starts manager nicholas bell giving welcome in...,0
3,must assumed praised nothe greatest filmed ope...,0
4,superbly trashy wondrously unpretentious explo...,1
...,...,...
24995,seems like consideration gone imdb reviews wen...,0
24996,not believe made completely unnecessary first ...,0
24997,guy loser not get girls needs build picked str...,0
24998,minute documentary buñuel made early spain poo...,0


In [8]:
pip install prettytable

Note: you may need to restart the kernel to use updated packages.


In [5]:
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from prettytable import PrettyTable
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

In [6]:
# lemmatization of word 
class LemmaTokenizer(object):
    def __init__(self):
        self.wordnetlemma = WordNetLemmatizer()
    def __call__(self, reviews):
        return [self.wordnetlemma.lemmatize(word) for word in word_tokenize(reviews)]

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Vectoization with Count Vectorizer and TFIDF Vectorizer with Unigram

In [17]:
train,test=train_test_split(df,test_size=.2,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,1),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews3']).toarray()
x_test_count = countvect.transform(test['Reviews3']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews3']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews3']).toarray()
y_train = train['Label']
y_test = test['Label']

C:\Users\anush\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### Feature importance with Logistic Regression and count vectorizer unigrams

In [19]:

lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+----------------+------------------------+
|    Feature     |         Score          |
+----------------+------------------------+
|      able      |  0.23915222157713895   |
|   absolutely   |  -0.10952913533300214  |
|      act       |  -0.01617946993576759  |
|     acting     |  -0.0717062261320832   |
|     action     |   0.1846975148962473   |
|     actor      |  -0.1521148398602906   |
|    actress     | -0.028773543130556786  |
|    actually    |  0.05921406236224636   |
|      add       |  0.15776267053154416   |
|      age       |  0.23077707241369272   |
|      ago       |  0.31121341755996956   |
|     almost     |  0.03594644459226185   |
|     alone      |  0.028110773391132333  |
|     along      |   0.2556341861923326   |
|    already     |  -0.17977290163116777  |
|      also      |  0.15839691847171455   |
|    although    |  0.24930009260608493   |
|     always     |  0.23769156206996636   |
|    amazing     |   0.8500274445534864   |
|    american    |  0.0440469687

### ### ### Feature importance with Logistic Regression and TFIDF unigrams

In [21]:
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=100:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+----------------+-----------------------+
|    Feature     |         Score         |
+----------------+-----------------------+
|      able      |   1.1281056157147933  |
|   absolutely   |  -0.4272619891008273  |
|      act       |  -0.5916233869302004  |
|     acting     |  -0.8231443762659363  |
|     action     |   1.2693880013628822  |
|     actor      |  -1.1317405348449885  |
|    actress     |  -0.22872331135731078 |
|    actually    |  0.03514235929495545  |
|      add       |   0.6566993801064176  |
|      age       |   1.3031621172496524  |
|      ago       |   0.8688814199919598  |
|     almost     |  0.19165407871950824  |
|     alone      |  0.07208248527139201  |
|     along      |   1.0496526469344771  |
|    already     |  -0.7048252322297271  |
|      also      |   1.9664310698996836  |
|    although    |   1.5374602326283358  |
|     always     |   1.9602895256203177  |
|    amazing     |   3.9781076499241474  |
|    american    |   0.4610667432496475  |
|    anothe

# Vectorization with Count Vectorizer and TFIDF Vectorizer with Bigram

In [22]:
train,test=train_test_split(df,test_size=.2,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(2,2), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(2,2),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews3']).toarray()
x_test_count = countvect.transform(test['Reviews3']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews3']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews3']).toarray()
y_train = train['Label']
y_test = test['Label']

C:\Users\anush\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### Feature Importance with Logistic Regression and Count Vectorizer with Bigram


In [23]:

lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+-----------------------+-----------------------+
|        Feature        |         Score         |
+-----------------------+-----------------------+
|   absolutely nothing  |   -1.300229904721879  |
|     academy award     |   0.609210288242368   |
|        act like       |  -0.3443346133943169  |
|       acting bad      |  -0.7496113608616027  |
|      acting good      |   0.5250062743184207  |
|       acting not      |  -0.3520627790153579  |
|      action scene     |  -0.20399489827552603 |
|    action sequence    | -0.048593294064832135 |
|     actor actress     |  0.16051227602587692  |
|       actor not       |  -0.1518222066569507  |
|      actually not     | -0.061227299119371595 |
|    african american   |   0.2519297061237181  |
|         ai not        |  -0.15501878548489462 |
|      almost every     |  0.10644944636670567  |
|       along way       |   0.4450958644489102  |
|       also good       |   1.2049902361681317  |
|        also not       |  -0.2690939028080096  |


### Feature Importance with Logistic Regression and TFIDF with Bigram

In [25]:
#with lgr tfidf bigrams
lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=100:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+-----------------------+-----------------------+
|        Feature        |         Score         |
+-----------------------+-----------------------+
|   absolutely nothing  |  -2.2697040251998826  |
|     academy award     |   1.3016815071288752  |
|        act like       |  -0.5356322556153775  |
|       acting bad      |  -2.1024192189248767  |
|      acting good      |   0.8187124962526401  |
|       acting not      |  -0.8855874892682989  |
|      action scene     |  -0.41529893445305777 |
|    action sequence    |  -0.2667003138232564  |
|     actor actress     |   0.3737294427166373  |
|       actor not       |  -0.41579085868850246 |
|      actually not     |  -0.07046495533030508 |
|    african american   |   0.4417895575502813  |
|         ai not        |  -0.23457797412607145 |
|      almost every     | -0.010080334099782847 |
|       along way       |   0.8194338392060134  |
|       also good       |   2.175627064756043   |
|        also not       |  -0.6726374813598597  |


# Vectorization with Count Vectorizer and TFIDF Vectorizer with Trigram

In [26]:
train,test=train_test_split(df,test_size=.2,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(3,3), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(3,3),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews3']).toarray()
x_test_count = countvect.transform(test['Reviews3']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews3']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews3']).toarray()
y_train = train['Label']
y_test = test['Label']

C:\Users\anush\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### Feature Importance with Logistic Regression and count vectorizer with Trigram

In [27]:

lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+--------------------------+-----------------------+
|         Feature          |         Score         |
+--------------------------+-----------------------+
|    academy award best    |   1.041685088627885   |
|     acting ever seen     |  -0.2942848283615027  |
|      acting not bad      |  -0.8481401103059463  |
|     acting not good      |  0.19702219411858446  |
|    acting pretty good    |  -0.03947688685261741 |
|  acting special effect   |  -0.3626356044887048  |
|    action take place     |   0.5373825341297817  |
|     actor ever seen      |  -1.6026364768631607  |
|     actor look like      |  -0.6010899385318832  |
|     actually not bad     |  -0.6084557276014866  |
|   actually pretty good   |  -0.34303187447206385 |
|    almost every scene    |  -0.5886215820207589  |
|    always ring twice     |   1.1536132369793848  |
|   attack killer tomato   |  -1.4864313759180896  |
|    avoid like plague     |   -2.400224949792681  |
|      bad acting bad      |  -1.5581375795448

### Feature Importance with Logistic Regression and TFIDF with Trigrams

In [28]:
#with lgr tfidf trigrams

lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=100:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+--------------------------+----------------------+
|         Feature          |        Score         |
+--------------------------+----------------------+
|    academy award best    |  1.1401444686130153  |
|     acting ever seen     | -0.3443703556919081  |
|      acting not bad      |  -1.100252446722327  |
|     acting not good      | 0.046316910362877196 |
|    acting pretty good    | -0.07619658733514649 |
|  acting special effect   | -0.35829988199110857 |
|    action take place     |  0.5402401439264995  |
|     actor ever seen      |  -1.724311363579455  |
|     actor look like      |  -0.75981224613241   |
|     actually not bad     | -0.6602500501295396  |
|   actually pretty good   | -0.30160956671417066 |
|    almost every scene    | -0.7045975852120075  |
|    always ring twice     |  1.3200546854476594  |
|   attack killer tomato   |  -1.660307049684307  |
|    avoid like plague     | -2.5888027009726544  |
|      bad acting bad      | -2.2252859945827255  |
|      bad a

# Vectorization with Count Vectorizer and TFIDF Vectorizer with 4-gram

In [7]:
train,test=train_test_split(df,test_size=.2,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(4,4), min_df=10,max_features=500)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(4,4),min_df=10,max_features=500)
x_train_count = countvect.fit_transform(train['Reviews3']).toarray()
x_test_count = countvect.transform(test['Reviews3']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews3']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews3']).toarray()
y_train = train['Label']
y_test = test['Label']

C:\Users\anush\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### Feature Importance with Logistic Regression and count vectorizer with 4-grams

In [9]:
#Feature Importance with Logistic Regression and Count Vectorizer with 4-gram

lgr = LogisticRegression()
lgr.fit(x_train_count,y_train)
lgr.score(x_test_count,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(countvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+-------------------------------+----------------------+
|            Feature            |        Score         |
+-------------------------------+----------------------+
| according dvd sleeve synopsis | -1.4564522587846804  |
|      best movie ever seen     |  1.6378785958694964  |
|     better luck next time     | -1.3681283736113297  |
| crouching tiger hidden dragon |  0.5452266733165421  |
|   fred astaire ginger rogers  |  0.6326670773312395  |
|       go ahead make day       |  1.7448162036858876  |
|        good guy bad guy       |  0.5452266733165422  |
|     jean claude van damme     | -1.6049012134500762  |
|    left cutting room floor    | -0.4554647343862889  |
|      let face music dance     |  1.2486878137228377  |
|      life never get back      | -2.0073703003723162  |
|    like watching paint dry    | -1.5989790217381983  |
|  lose friend alienate people  |  1.5131795575376585  |
|       low budget sci fi       | -0.16106337357126113 |
|      movie seen long time    

### Feature Importance with Logistic Regression and TFIDF with 4-grams

In [11]:

lgr = LogisticRegression()
lgr.fit(x_train_tfidf,y_train)
lgr.score(x_test_tfidf,y_test)
lgr.coef_[0]
i=0
importantfeature = PrettyTable(["Feature", "Score"])
for feature, importance in zip(tfidfvect.get_feature_names_out(), lgr.coef_[0]):
    if i<=200:
        importantfeature.add_row([feature, importance])
        i=i+1
print(importantfeature)

+-------------------------------+----------------------+
|            Feature            |        Score         |
+-------------------------------+----------------------+
| according dvd sleeve synopsis | -1.4687220723542866  |
|      best movie ever seen     |  1.614736580777274   |
|     better luck next time     | -1.3898983688740492  |
| crouching tiger hidden dragon |  0.5370101182973015  |
|   fred astaire ginger rogers  |  0.6617228768799405  |
|       go ahead make day       |  1.7694878956948317  |
|        good guy bad guy       |  0.5370101182973015  |
|     jean claude van damme     | -1.5955503634169794  |
|    left cutting room floor    | -0.4592567198837547  |
|      let face music dance     |  1.3438509182251215  |
|      life never get back      |  -2.014400757530216  |
|    like watching paint dry    |  -1.64386379407902   |
|  lose friend alienate people  |  1.6993513593989387  |
|       low budget sci fi       | -0.16419316450189128 |
|      movie seen long time    

# Vectorization with Count Vectorizer and TDIDF Vectorizer with unigram, bigram and trigram


In [12]:
#Vectorization with Count Vectorizer and TDIDF Vectorizer with unigram, bigram and trigram

train,test=train_test_split(df,test_size=.2,random_state=42, shuffle=True)
countvect = CountVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3), min_df=10,max_features=5000)
tfidfvect = TfidfVectorizer(analyzer = "word", tokenizer = LemmaTokenizer(), ngram_range=(1,3),min_df=10,max_features=5000)
x_train_count = countvect.fit_transform(train['Reviews3']).toarray()
x_test_count = countvect.transform(test['Reviews3']).toarray()
x_train_tfidf = tfidfvect.fit_transform(train['Reviews3']).toarray()
x_test_tfidf = tfidfvect.transform(test['Reviews3']).toarray()
y_train = train['Label']
y_test = test['Label']

C:\Users\anush\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# Feature Selection with Chi squared


In [13]:
from sklearn.feature_selection import chi2
import numpy as np
N = 500
Number = 1
featureselection = PrettyTable(["Unigram", "Bigram","Trigram"])
for category in train['Label'].unique():
    features_chi2 = chi2(x_train_tfidf, train['Label'] == category)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidfvect.get_feature_names_out())[indices]
    unigrams = [x for x in feature_names if len(x.split(' ')) == 1]
    bigrams = [x for x in feature_names if len(x.split(' ')) == 2]
    trigrams = [x for x in feature_names if len(x.split(' ')) == 3]
    print("%s. %s :" % (Number,category))
    print("\t# Unigrams :\n\t. %s" %('\n\t. '.join(unigrams[-N:])))
    print("\t# Bigrams :\n\t. %s" %('\n\t. '.join(bigrams[-N:])))
    print("\t# Trigrams :\n\t. %s" %('\n\t. '.join(trigrams[-N:])))
    Number += 1

1. 0 :
	# Unigrams :
	. marie
	. dead
	. bond
	. torture
	. contrived
	. awake
	. human
	. sirk
	. soprano
	. soccer
	. dream
	. disjointed
	. porno
	. lemmon
	. although
	. supposedly
	. rukh
	. scientist
	. effect
	. italy
	. available
	. halfway
	. edge
	. son
	. shallow
	. creates
	. useless
	. cried
	. fat
	. jim
	. warm
	. wanted
	. widmark
	. ratso
	. unlike
	. mildly
	. moron
	. bakshi
	. wreck
	. surprisingly
	. elvira
	. endless
	. incomprehensible
	. disappointed
	. animated
	. winning
	. confusing
	. plastic
	. frank
	. fresh
	. godzilla
	. recommend
	. basically
	. deeply
	. exceptional
	. gorgeous
	. alien
	. brings
	. andrew
	. cut
	. worthless
	. satisfying
	. ritter
	. god
	. sullivan
	. skull
	. show
	. dance
	. haunting
	. killing
	. miscast
	. throw
	. paint
	. nicely
	. fuller
	. neither
	. treat
	. stanwyck
	. memorable
	. crocodile
	. hell
	. sensitive
	. scary
	. harry
	. accent
	. flynn
	. tiresome
	. point
	. inane
	. else
	. original
	. crafted
	. ham
	. obno

2. 1 :
	# Unigrams :
	. marie
	. dead
	. bond
	. torture
	. contrived
	. awake
	. human
	. sirk
	. soprano
	. soccer
	. dream
	. disjointed
	. porno
	. lemmon
	. although
	. supposedly
	. rukh
	. scientist
	. effect
	. italy
	. available
	. halfway
	. edge
	. son
	. shallow
	. creates
	. useless
	. cried
	. fat
	. jim
	. warm
	. wanted
	. widmark
	. ratso
	. unlike
	. mildly
	. moron
	. bakshi
	. wreck
	. surprisingly
	. elvira
	. endless
	. incomprehensible
	. disappointed
	. animated
	. winning
	. confusing
	. plastic
	. frank
	. fresh
	. godzilla
	. recommend
	. basically
	. deeply
	. exceptional
	. gorgeous
	. alien
	. brings
	. andrew
	. cut
	. worthless
	. satisfying
	. ritter
	. god
	. sullivan
	. skull
	. show
	. dance
	. haunting
	. killing
	. miscast
	. throw
	. paint
	. nicely
	. fuller
	. neither
	. treat
	. stanwyck
	. memorable
	. crocodile
	. hell
	. sensitive
	. scary
	. harry
	. accent
	. flynn
	. tiresome
	. point
	. inane
	. else
	. original
	. crafted
	. ham
	. obno

# Model selection

In [14]:
import sys
import numpy as np
import scipy as sp
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score,accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

In [15]:
model_1=LogisticRegression()

In [16]:
%%time
model_1.fit(x_train_tfidf,y_train)

CPU times: total: 2.31 s
Wall time: 1.53 s


LogisticRegression()

In [17]:
print("Accuracy on training dataset for Logistic Regression: %s" % accuracy_score(y_train,model_1.predict(x_train_tfidf)))
print("Precision Score on training dateset for Logistic Regression: %s" % precision_score(y_train,model_1.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for Logistic Regression: %s" % roc_auc_score(y_train,model_1.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_1 =f1_score(y_train,model_1.predict(x_train_tfidf),average="weighted")
print("F1 Score ftraining dateset for Logistic Regression: %s" % f1_score_train_1)
print("Accuracy on test dataset for Logistic Regression: %s" % accuracy_score(y_test,model_1.predict(x_test_tfidf)))
print("Precision Score on test for Logistic Regression: %s" % precision_score(y_test,model_1.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for Logistic Regression: %s" % roc_auc_score(y_test,model_1.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_1 =f1_score(y_test,model_1.predict(x_test_tfidf),average="weighted")
print("F1 Score for Logistic Regression: %s" % f1_score_1)

Accuracy on training dataset for Logistic Regression: 0.9195
Precision Score on training dateset for Logistic Regression: 0.9195
AUC Score on training dateset for Logistic Regression: 0.9746831386061304
F1 Score ftraining dateset for Logistic Regression: 0.9194939089994235
Accuracy on test dataset for Logistic Regression: 0.8906
Precision Score on test for Logistic Regression: 0.8906
AUC Score on test for Logistic Regression: 0.9555950351692314
F1 Score for Logistic Regression: 0.8905801310407189


In [16]:
## Decision Tree

In [17]:

model_2 = Pipeline(
    steps=[
    ("classifier", DecisionTreeClassifier())
    ]
)

In [18]:
model_2.fit(x_train_tfidf,y_train)


Pipeline(steps=[('classifier', DecisionTreeClassifier())])

In [19]:
print("Accuracy on training dataset for Decision Tree: %s" % accuracy_score(y_train,model_2.predict(x_train_tfidf)))
print("Precision Score on training dateset for Decision Tree Classifier: %s" % precision_score(y_train,model_2.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for Decision Tree Classifier: %s" % roc_auc_score(y_train,model_2.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_2 =f1_score(y_train,model_2.predict(x_train_tfidf),average="weighted")
print("F1 Score training dateset for Decision Tree Classifier: %s" % f1_score_train_2)
print("Accuracy on test dataset for Decision Tree: %s" % accuracy_score(y_test,model_2.predict(x_test_tfidf)))
print("Precision Score on test for Decision Tree Classifier: %s" % precision_score(y_test,model_2.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for Decision Tree Classifier: %s" % roc_auc_score(y_test,model_2.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_2 =f1_score(y_test,model_2.predict(x_test_tfidf),average="weighted")
print("F1 Score for Decision Tree Classifier: %s" % f1_score_2)

Accuracy on training dataset for Decision Tree: 1.0
Precision Score on training dateset for Decision Tree Classifier: 1.0
AUC Score on training dateset for Decision Tree Classifier: 1.0
F1 Score training dateset for Decision Tree Classifier: 1.0
Accuracy on test dataset for Decision Tree: 0.7092
Precision Score on test for Decision Tree Classifier: 0.7092
AUC Score on test for Decision Tree Classifier: 0.7092804880409892
F1 Score for Decision Tree Classifier: 0.7091806438286091


In [20]:
#FIXING DECISION TREE OVERFITTING WITH MAX DEPTH 11

model_3 = Pipeline(
    steps=[
        ("classifier", DecisionTreeClassifier( criterion='gini', max_depth=11, min_samples_split=2, min_samples_leaf=1)),
    ]
)

In [21]:
model_3.fit(x_train_tfidf,y_train)

Pipeline(steps=[('classifier', DecisionTreeClassifier(max_depth=11))])

In [22]:
print("Accuracy on train dataset for decision Tree: %s" % accuracy_score(y_train,model_3.predict(x_train_tfidf)))
print("Precision Score on training dateset for Decision Tree Classifier: %s" % precision_score(y_train,model_3.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for Decision Tree Classifier: %s" % roc_auc_score(y_train,model_3.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_3 =f1_score(y_train,model_3.predict(x_train_tfidf),average="weighted")
print("F1 Score training dateset for Decision Tree Classifier: %s" % f1_score_train_3)
print("Accuracy on test dataset for Decision Tree: %s" % accuracy_score(y_test,model_3.predict(x_test_tfidf)))
print("Precision Score on test for Decision Tree Classifier: %s" % precision_score(y_test,model_3.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for Decision Tree Classifier: %s" % roc_auc_score(y_test,model_3.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_3 =f1_score(y_test,model_3.predict(x_test_tfidf),average="weighted")
print("F1 Score for Decision Tree Classifier: %s" % f1_score_3)

Accuracy on train dataset for decision Tree: 0.78605
Precision Score on training dateset for Decision Tree Classifier: 0.78605
AUC Score on training dateset for Decision Tree Classifier: 0.8498202428510767
F1 Score training dateset for Decision Tree Classifier: 0.7836563859346115
Accuracy on test dataset for Decision Tree: 0.7306
Precision Score on test for Decision Tree Classifier: 0.7306
AUC Score on test for Decision Tree Classifier: 0.7667360466740559
F1 Score for Decision Tree Classifier: 0.7277468472802393


In [23]:
model_4 = Pipeline(
    steps=[
    ("classifier", RandomForestClassifier())]
)

In [24]:
model_4.fit(x_train_tfidf,y_train)

Pipeline(steps=[('classifier', RandomForestClassifier())])

In [25]:
print("Accuracy on train dataset for Random Forest Classifier: %s" % accuracy_score(y_train,model_4.predict(x_train_tfidf)))
print("Precision Score on training dateset for Random Forest Classifier: %s" % precision_score(y_train,model_4.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for Random Forest Classifier: %s" % roc_auc_score(y_train,model_4.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_4 =f1_score(y_train,model_4.predict(x_train_tfidf),average="weighted")
print("F1 Score training dateset for Random Forest Classifier: %s" % f1_score_train_4)
print("Accuracy on test dataset for Random Forest Classifier: %s" % accuracy_score(y_test,model_4.predict(x_test_tfidf)))
print("Precision Score on test for Random Forest Classifier: %s" % precision_score(y_test,model_4.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for Random Forest Classifier: %s" % roc_auc_score(y_test,model_4.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_4 =f1_score(y_test,model_4.predict(x_test_tfidf),average="weighted")
print("F1 Score for Random Forest Classifier: %s" % f1_score_4)

Accuracy on train dataset for Random Forest Classifier: 1.0
Precision Score on training dateset for Random Forest Classifier: 1.0
AUC Score on training dateset for Random Forest Classifier: 1.0
F1 Score training dateset for Random Forest Classifier: 1.0
Accuracy on test dataset for Random Forest Classifier: 0.8562
Precision Score on test for Random Forest Classifier: 0.8562
AUC Score on test for Random Forest Classifier: 0.9275175734150405
F1 Score for Random Forest Classifier: 0.8562015012768641


In [26]:
#FIXING OVERFITTING OF RANDOM FOREST
model_5 = Pipeline(
    steps=[
    ("classifier", RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=15, min_samples_split=3, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='sqrt', max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None))
    ]
)


In [27]:
model_5.fit(x_train_tfidf,y_train)

Pipeline(steps=[('classifier',
                 RandomForestClassifier(max_depth=15, min_samples_split=3))])

In [28]:
print("Accuracy on train dataset for Random forest classifier: %s" % accuracy_score(y_train,model_5.predict(x_train_tfidf)))
print("Precision Score on training dateset for Random Forest Classifier: %s" % precision_score(y_train,model_5.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for Random Forest Classifier: %s" % roc_auc_score(y_train,model_5.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_4 =f1_score(y_train,model_5.predict(x_train_tfidf),average="weighted")
print("F1 Score training dateset for Random Forest Classifier: %s" % f1_score_train_4)
print("Accuracy on test dataset for Random forest classifier: %s" % accuracy_score(y_test,model_5.predict(x_test_tfidf)))
print("Precision Score on test for Random Forest Classifier: %s" % precision_score(y_test,model_5.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for Random Forest Classifier: %s" % roc_auc_score(y_test,model_5.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_4 =f1_score(y_test,model_5.predict(x_test_tfidf),average="weighted")
print("F1 Score for Random Forest Classifier: %s" % f1_score_4)

Accuracy on train dataset for Random forest classifier: 0.90045
Precision Score on training dateset for Random Forest Classifier: 0.90045
AUC Score on training dateset for Random Forest Classifier: 0.969080988382368
F1 Score training dateset for Random Forest Classifier: 0.9002114965205789
Accuracy on test dataset for Random forest classifier: 0.8354
Precision Score on test for Random Forest Classifier: 0.8354
AUC Score on test for Random Forest Classifier: 0.9153221490073267
F1 Score for Random Forest Classifier: 0.8350093108569064


In [29]:
model_6 = Pipeline(
    steps=[
    ("classifier", MultinomialNB())
    ]
)

In [30]:
model_6.fit(x_train_tfidf,y_train)

Pipeline(steps=[('classifier', MultinomialNB())])

In [31]:
print("Accuracy on train dataset for NP: %s" % accuracy_score(y_train,model_6.predict(x_train_tfidf)))
print("Precision Score on training dataset for NP: %s" % precision_score(y_train,model_6.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for NP: %s" % roc_auc_score(y_train,model_6.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_4 =f1_score(y_train,model_6.predict(x_train_tfidf),average="weighted")
print("F1 Score training dateset for NP: %s" % f1_score_train_4)
print("Accuracy on test dataset for NP: %s" % accuracy_score(y_test,model_6.predict(x_test_tfidf)))
print("Precision Score on test for NP: %s" % precision_score(y_test,model_6.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for NP: %s" % roc_auc_score(y_test,model_6.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_4 =f1_score(y_test,model_6.predict(x_test_tfidf),average="weighted")
print("F1 Score for NP: %s" % f1_score_4)

Accuracy on train dataset for NP: 0.8734
Precision Score on training dataset for NP: 0.8734
AUC Score on training dateset for NP: 0.9455296233619404
F1 Score training dateset for NP: 0.8733792068034755
Accuracy on test dataset for NP: 0.8582
Precision Score on test for NP: 0.8582
AUC Score on test for NP: 0.933835058312968
F1 Score for NP: 0.8581406633318218


# Hyperparamter Tuning

In [32]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection

def hyperparamtune(classifier, param_grid,metric,verbose_value,cv):
    model=model_selection.GridSearchCV(
            estimator=classifier,
            param_grid=param_grid,
            scoring=metric,
            verbose=verbose_value,            
            cv=cv)

    model.fit(x_train_tfidf,y_train)
    print("Best Score %s" % {model.best_score_})
    print("Best hyperparameter set:")
    best_parameters = model.best_estimator_.get_params()
    for param_name in sorted(param_grid.keys()):
        print(f"\t{param_name}: {best_parameters[param_name]}")
    return model, best_parameters

In [33]:
%%time
param_gd = {
    "penalty": ["l2", "l1"],  
    "C": [0.01,0.1, 1.0, 10],  
    "tol": [0.0001,0.001, 0.01,0.1,10], 
    "max_iter": [100, 200, 300]  
}

model_7, best_param = hyperparamtune(LogisticRegression(), param_gd, "accuracy", 10, 5)


Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5; 1/120] START C=0.01, max_iter=100, penalty=l2, tol=0.0001..............
[CV 1/5; 1/120] END C=0.01, max_iter=100, penalty=l2, tol=0.0001;, score=0.839 total time=   0.9s
[CV 2/5; 1/120] START C=0.01, max_iter=100, penalty=l2, tol=0.0001..............
[CV 2/5; 1/120] END C=0.01, max_iter=100, penalty=l2, tol=0.0001;, score=0.826 total time=   1.1s
[CV 3/5; 1/120] START C=0.01, max_iter=100, penalty=l2, tol=0.0001..............
[CV 3/5; 1/120] END C=0.01, max_iter=100, penalty=l2, tol=0.0001;, score=0.844 total time=   1.0s
[CV 4/5; 1/120] START C=0.01, max_iter=100, penalty=l2, tol=0.0001..............
[CV 4/5; 1/120] END C=0.01, max_iter=100, penalty=l2, tol=0.0001;, score=0.830 total time=   1.0s
[CV 5/5; 1/120] START C=0.01, max_iter=100, penalty=l2, tol=0.0001..............
[CV 5/5; 1/120] END C=0.01, max_iter=100, penalty=l2, tol=0.0001;, score=0.835 total time=   0.9s
[CV 1/5; 2/120] START C=0.01, max_iter=100

[CV 2/5; 10/120] END C=0.01, max_iter=100, penalty=l1, tol=10;, score=nan total time=   0.1s
[CV 3/5; 10/120] START C=0.01, max_iter=100, penalty=l1, tol=10.................
[CV 3/5; 10/120] END C=0.01, max_iter=100, penalty=l1, tol=10;, score=nan total time=   0.1s
[CV 4/5; 10/120] START C=0.01, max_iter=100, penalty=l1, tol=10.................
[CV 4/5; 10/120] END C=0.01, max_iter=100, penalty=l1, tol=10;, score=nan total time=   0.1s
[CV 5/5; 10/120] START C=0.01, max_iter=100, penalty=l1, tol=10.................
[CV 5/5; 10/120] END C=0.01, max_iter=100, penalty=l1, tol=10;, score=nan total time=   0.2s
[CV 1/5; 11/120] START C=0.01, max_iter=200, penalty=l2, tol=0.0001.............
[CV 1/5; 11/120] END C=0.01, max_iter=200, penalty=l2, tol=0.0001;, score=0.839 total time=   0.9s
[CV 2/5; 11/120] START C=0.01, max_iter=200, penalty=l2, tol=0.0001.............
[CV 2/5; 11/120] END C=0.01, max_iter=200, penalty=l2, tol=0.0001;, score=0.826 total time=   0.8s
[CV 3/5; 11/120] START C=

[CV 4/5; 19/120] END C=0.01, max_iter=200, penalty=l1, tol=0.1;, score=nan total time=   0.1s
[CV 5/5; 19/120] START C=0.01, max_iter=200, penalty=l1, tol=0.1................
[CV 5/5; 19/120] END C=0.01, max_iter=200, penalty=l1, tol=0.1;, score=nan total time=   0.2s
[CV 1/5; 20/120] START C=0.01, max_iter=200, penalty=l1, tol=10.................
[CV 1/5; 20/120] END C=0.01, max_iter=200, penalty=l1, tol=10;, score=nan total time=   0.2s
[CV 2/5; 20/120] START C=0.01, max_iter=200, penalty=l1, tol=10.................
[CV 2/5; 20/120] END C=0.01, max_iter=200, penalty=l1, tol=10;, score=nan total time=   0.1s
[CV 3/5; 20/120] START C=0.01, max_iter=200, penalty=l1, tol=10.................
[CV 3/5; 20/120] END C=0.01, max_iter=200, penalty=l1, tol=10;, score=nan total time=   0.2s
[CV 4/5; 20/120] START C=0.01, max_iter=200, penalty=l1, tol=10.................
[CV 4/5; 20/120] END C=0.01, max_iter=200, penalty=l1, tol=10;, score=nan total time=   0.1s
[CV 5/5; 20/120] START C=0.01, max_

[CV 1/5; 29/120] START C=0.01, max_iter=300, penalty=l1, tol=0.1................
[CV 1/5; 29/120] END C=0.01, max_iter=300, penalty=l1, tol=0.1;, score=nan total time=   0.1s
[CV 2/5; 29/120] START C=0.01, max_iter=300, penalty=l1, tol=0.1................
[CV 2/5; 29/120] END C=0.01, max_iter=300, penalty=l1, tol=0.1;, score=nan total time=   0.2s
[CV 3/5; 29/120] START C=0.01, max_iter=300, penalty=l1, tol=0.1................
[CV 3/5; 29/120] END C=0.01, max_iter=300, penalty=l1, tol=0.1;, score=nan total time=   0.1s
[CV 4/5; 29/120] START C=0.01, max_iter=300, penalty=l1, tol=0.1................
[CV 4/5; 29/120] END C=0.01, max_iter=300, penalty=l1, tol=0.1;, score=nan total time=   0.2s
[CV 5/5; 29/120] START C=0.01, max_iter=300, penalty=l1, tol=0.1................
[CV 5/5; 29/120] END C=0.01, max_iter=300, penalty=l1, tol=0.1;, score=nan total time=   0.2s
[CV 1/5; 30/120] START C=0.01, max_iter=300, penalty=l1, tol=10.................
[CV 1/5; 30/120] END C=0.01, max_iter=300, p

[CV 3/5; 38/120] END C=0.1, max_iter=100, penalty=l1, tol=0.01;, score=nan total time=   0.1s
[CV 4/5; 38/120] START C=0.1, max_iter=100, penalty=l1, tol=0.01................
[CV 4/5; 38/120] END C=0.1, max_iter=100, penalty=l1, tol=0.01;, score=nan total time=   0.1s
[CV 5/5; 38/120] START C=0.1, max_iter=100, penalty=l1, tol=0.01................
[CV 5/5; 38/120] END C=0.1, max_iter=100, penalty=l1, tol=0.01;, score=nan total time=   0.2s
[CV 1/5; 39/120] START C=0.1, max_iter=100, penalty=l1, tol=0.1.................
[CV 1/5; 39/120] END C=0.1, max_iter=100, penalty=l1, tol=0.1;, score=nan total time=   0.1s
[CV 2/5; 39/120] START C=0.1, max_iter=100, penalty=l1, tol=0.1.................
[CV 2/5; 39/120] END C=0.1, max_iter=100, penalty=l1, tol=0.1;, score=nan total time=   0.1s
[CV 3/5; 39/120] START C=0.1, max_iter=100, penalty=l1, tol=0.1.................
[CV 3/5; 39/120] END C=0.1, max_iter=100, penalty=l1, tol=0.1;, score=nan total time=   0.1s
[CV 4/5; 39/120] START C=0.1, max_

...

[CV 1/5; 95/120] END C=10, max_iter=100, penalty=l2, tol=10;, score=0.501 total time=   0.4s
[CV 2/5; 95/120] START C=10, max_iter=100, penalty=l2, tol=10...................
[CV 2/5; 95/120] END C=10, max_iter=100, penalty=l2, tol=10;, score=0.501 total time=   0.5s
[CV 3/5; 95/120] START C=10, max_iter=100, penalty=l2, tol=10...................
[CV 3/5; 95/120] END C=10, max_iter=100, penalty=l2, tol=10;, score=0.501 total time=   0.5s
[CV 4/5; 95/120] START C=10, max_iter=100, penalty=l2, tol=10...................
[CV 4/5; 95/120] END C=10, max_iter=100, penalty=l2, tol=10;, score=0.501 total time=   0.4s
[CV 5/5; 95/120] START C=10, max_iter=100, penalty=l2, tol=10...................
[CV 5/5; 95/120] END C=10, max_iter=100, penalty=l2, tol=10;, score=0.501 total time=   0.4s
[CV 1/5; 96/120] START C=10, max_iter=100, penalty=l1, tol=0.0001...............
[CV 1/5; 96/120] END C=10, max_iter=100, penalty=l1, tol=0.0001;, score=nan total time=   0.1s
[CV 2/5; 96/120] START C=10, max_it

[CV 3/5; 104/120] END C=10, max_iter=200, penalty=l2, tol=0.1;, score=0.501 total time=   0.4s
[CV 4/5; 104/120] START C=10, max_iter=200, penalty=l2, tol=0.1.................
[CV 4/5; 104/120] END C=10, max_iter=200, penalty=l2, tol=0.1;, score=0.501 total time=   0.4s
[CV 5/5; 104/120] START C=10, max_iter=200, penalty=l2, tol=0.1.................
[CV 5/5; 104/120] END C=10, max_iter=200, penalty=l2, tol=0.1;, score=0.501 total time=   0.4s
[CV 1/5; 105/120] START C=10, max_iter=200, penalty=l2, tol=10..................
[CV 1/5; 105/120] END C=10, max_iter=200, penalty=l2, tol=10;, score=0.501 total time=   0.4s
[CV 2/5; 105/120] START C=10, max_iter=200, penalty=l2, tol=10..................
[CV 2/5; 105/120] END C=10, max_iter=200, penalty=l2, tol=10;, score=0.501 total time=   0.4s
[CV 3/5; 105/120] START C=10, max_iter=200, penalty=l2, tol=10..................
[CV 3/5; 105/120] END C=10, max_iter=200, penalty=l2, tol=10;, score=0.501 total time=   0.5s
[CV 4/5; 105/120] START C=10

[CV 5/5; 113/120] END C=10, max_iter=300, penalty=l2, tol=0.01;, score=0.501 total time=   0.4s
[CV 1/5; 114/120] START C=10, max_iter=300, penalty=l2, tol=0.1.................
[CV 1/5; 114/120] END C=10, max_iter=300, penalty=l2, tol=0.1;, score=0.501 total time=   0.4s
[CV 2/5; 114/120] START C=10, max_iter=300, penalty=l2, tol=0.1.................
[CV 2/5; 114/120] END C=10, max_iter=300, penalty=l2, tol=0.1;, score=0.501 total time=   0.4s
[CV 3/5; 114/120] START C=10, max_iter=300, penalty=l2, tol=0.1.................
[CV 3/5; 114/120] END C=10, max_iter=300, penalty=l2, tol=0.1;, score=0.501 total time=   0.4s
[CV 4/5; 114/120] START C=10, max_iter=300, penalty=l2, tol=0.1.................
[CV 4/5; 114/120] END C=10, max_iter=300, penalty=l2, tol=0.1;, score=0.501 total time=   0.4s
[CV 5/5; 114/120] START C=10, max_iter=300, penalty=l2, tol=0.1.................
[CV 5/5; 114/120] END C=10, max_iter=300, penalty=l2, tol=0.1;, score=0.501 total time=   0.5s
[CV 1/5; 115/120] START 

C:\Users\anush\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
300 fits failed out of a total of 600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
300 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\anush\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\anush\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\anush\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1194, in fit
    solver 

Best Score {0.8839}
Best hyperparameter set:
	C: 10
	max_iter: 100
	penalty: l2
	tol: 0.001
CPU times: total: 6min 11s
Wall time: 5min 42s


In [35]:
print("Accuracy on training dataset for Logistic Regression: %s" % accuracy_score(y_train,model_7.predict(x_train_tfidf)))
print("Precision Score on training dateset for Logistic Regression: %s" % precision_score(y_train,model_7.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for Logistic Regression: %s" % roc_auc_score(y_train,model_7.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_1 =f1_score(y_train,model_7.predict(x_train_tfidf),average="weighted")
print("F1 Score ftraining dateset for Logistic Regression: %s" % f1_score_train_1)
print("Accuracy on test dataset for Logistic Regression: %s" % accuracy_score(y_test,model_7.predict(x_test_tfidf)))
print("Precision Score on test for Logistic Regression: %s" % precision_score(y_test,model_7.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for Logistic Regression: %s" % roc_auc_score(y_test,model_7.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_1 =f1_score(y_test,model_7.predict(x_test_tfidf),average="weighted")
print("F1 Score for Logistic Regression: %s" % f1_score_1)

Accuracy on training dataset for Logistic Regression: 0.92945
Precision Score on training dateset for Logistic Regression: 0.92945
AUC Score on training dateset for Logistic Regression: 0.9794635258633283
F1 Score ftraining dateset for Logistic Regression: 0.9294486978078755
Accuracy on test dataset for Logistic Regression: 0.8892
Precision Score on test for Logistic Regression: 0.8892
AUC Score on test for Logistic Regression: 0.9563689998734326
F1 Score for Logistic Regression: 0.8891918082366672


In [62]:
%%time
param_gd = {
    "n_estimators": [100, 200],
    "max_depth": [11, 17],
    "criterion": ["gini", "entropy"],
    "min_samples_split": [3],
    "min_samples_leaf": [3],
    "max_features": ["sqrt"]
}

model_8, best_param_10 = hyperparamtune(RandomForestClassifier(), param_gd, "accuracy", 10, 5)



Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5; 1/8] START criterion=gini, max_depth=11, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100
[CV 1/5; 1/8] END criterion=gini, max_depth=11, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100;, score=0.821 total time=  30.0s
[CV 2/5; 1/8] START criterion=gini, max_depth=11, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100
[CV 2/5; 1/8] END criterion=gini, max_depth=11, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100;, score=0.809 total time=  27.7s
[CV 3/5; 1/8] START criterion=gini, max_depth=11, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100
[CV 3/5; 1/8] END criterion=gini, max_depth=11, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100;, score=0.835 total time=  28.8s
[CV 4/5; 1/8] START criterion=gini, max_depth=11, max_features=sqrt, min_samples_leaf

[CV 4/5; 6/8] END criterion=entropy, max_depth=11, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=200;, score=0.823 total time=  57.7s
[CV 5/5; 6/8] START criterion=entropy, max_depth=11, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=200
[CV 5/5; 6/8] END criterion=entropy, max_depth=11, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=200;, score=0.826 total time=  58.0s
[CV 1/5; 7/8] START criterion=entropy, max_depth=17, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100
[CV 1/5; 7/8] END criterion=entropy, max_depth=17, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100;, score=0.823 total time=  41.5s
[CV 2/5; 7/8] START criterion=entropy, max_depth=17, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100
[CV 2/5; 7/8] END criterion=entropy, max_depth=17, max_features=sqrt, min_samples_leaf=3, min_samples_split=3, n_estimators=100

In [63]:
print("Accuracy on train dataset for Random Forest Classifier: %s" % accuracy_score(y_train,model_8.predict(x_train_tfidf)))
print("Precision Score on training dateset for Random Forest Classifier: %s" % precision_score(y_train,model_8.predict(x_train_tfidf),average='micro'))
print("AUC Score on training dateset for Random Forest Classifier: %s" % roc_auc_score(y_train,model_8.predict_proba(x_train_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_train_8 =f1_score(y_train,model_8.predict(x_train_tfidf),average="weighted")
print("F1 Score training dateset for Random Forest Classifier: %s" % f1_score_train_8)
print("Accuracy on test dataset for Random Forest Classifier: %s" % accuracy_score(y_test,model_8.predict(x_test_tfidf)))
print("Precision Score on test for Random Forest Classifier: %s" % precision_score(y_test,model_8.predict(x_test_tfidf),average='micro'))
print("AUC Score on test for Random Forest Classifier: %s" % roc_auc_score(y_test,model_8.predict_proba(x_test_tfidf)[:,1],multi_class='ovo',average='macro'))
f1_score_8 =f1_score(y_test,model_8.predict(x_test_tfidf),average="weighted")
print("F1 Score for Random Forest Classifier: %s" % f1_score_8)

Accuracy on train dataset for Random Forest Classifier: 0.89345
Precision Score on training dateset for Random Forest Classifier: 0.89345
AUC Score on training dateset for Random Forest Classifier: 0.9650093236836584
F1 Score training dateset for Random Forest Classifier: 0.8931947258088164
Accuracy on test dataset for Random Forest Classifier: 0.8376
Precision Score on test for Random Forest Classifier: 0.8376
AUC Score on test for Random Forest Classifier: 0.9193262202824836
F1 Score for Random Forest Classifier: 0.8371446945312848


In [19]:
import pickle


with open('modelreg.pkl', 'wb') as file:
    pickle.dump(model_1, file)
